# 02 - Hooks and Exit Conditions

This notebook demonstrates runtime controls you can adapt from this repo:
- `session_start` hook injects context
- `pre_tool_use` can allow/deny a tool call
- `post_tool_use` can append corrective context
- `stop` hook can block completion and force another loop turn


In [7]:
import json
import random
import re
import sys
from pathlib import Path
from typing import Any

# Ensure repo root is importable when running this notebook from its folder.
repo_root = Path.cwd()
while repo_root != repo_root.parent and not (repo_root / "orchestrator").exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from orchestrator.llm_interaction.adapter import LLMAdapter

MODEL = "llama3.1:8b"
adapter = LLMAdapter(model=MODEL)


In [8]:
world_state = {
    "location": "Moonfall Keep",
    "time": "20:00",
    "threat_level": "medium",
}

def get_world_state() -> dict[str, Any]:
    return world_state

def advance_clock(hours: int) -> dict[str, Any]:
    # simple toy clock update
    h, m = map(int, world_state["time"].split(":"))
    h = (h + hours) % 24
    world_state["time"] = f"{h:02d}:{m:02d}"
    return world_state

def roll_check(dc: int, modifier: int = 0) -> dict[str, Any]:
    roll = random.randint(1, 20)
    total = roll + modifier
    return {"roll": roll, "modifier": modifier, "total": total, "dc": dc, "success": total >= dc}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_world_state",
            "description": "Read current world state.",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "advance_clock",
            "description": "Advance world clock by N hours.",
            "parameters": {
                "type": "object",
                "properties": {"hours": {"type": "integer", "minimum": 1, "maximum": 24}},
                "required": ["hours"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "roll_check",
            "description": "Roll 1d20 plus modifier against a DC.",
            "parameters": {
                "type": "object",
                "properties": {
                    "dc": {"type": "integer"},
                    "modifier": {"type": "integer", "default": 0}
                },
                "required": ["dc"],
            },
        },
    },
]

TOOL_IMPL = {
    "get_world_state": get_world_state,
    "advance_clock": advance_clock,
    "roll_check": roll_check,
}


In [9]:
class DMHooks:
    def session_start(self, state: dict[str, Any]) -> str | None:
        return (
            "Hook context: keep continuity with prior events and keep danger proportional to threat_level. "
            f"Current threat_level is {state.get('threat_level')}."
        )

    def pre_tool_use(self, tool_name: str, args: dict[str, Any], state: dict[str, Any]) -> dict[str, Any]:
        # Example hard guard: disallow huge time skips in one move
        if tool_name == "advance_clock" and int(args.get("hours", 0)) > 8:
            return {"allow": False, "reason": "Do not skip more than 8 hours in a single turn."}
        return {"allow": True}

    def post_tool_use(self, tool_name: str, tool_result: dict[str, Any], state: dict[str, Any]) -> str | None:
        if not tool_result.get("ok", True):
            return "A tool failed. Recover by explaining the constraint and offering alternatives."
        return None

    def stop(self, assistant_text: str, state: dict[str, Any], stop_hook_active: bool) -> str | None:
        # Require clear user choices before ending turn
        text = (assistant_text or "").lower()
        has_choices = ("what do you do" in text) or ("choice" in text) or ("options" in text)
        if not has_choices:
            return "Before completing, provide 2-3 concrete player choices."

        # Optional second-pass stricter rule
        if stop_hook_active and len(assistant_text.strip()) < 80:
            return "Your response is too brief. Provide richer scene detail before ending."

        return None


hooks = DMHooks()


In [10]:
def execute_tool(name: str, arguments: dict[str, Any]) -> dict[str, Any]:
    fn = TOOL_IMPL.get(name)
    if not fn:
        return {"ok": False, "error": f"Unknown tool: {name}"}
    try:
        return {"ok": True, "result": fn(**arguments)}
    except Exception as e:
        return {"ok": False, "error": str(e)}


In [11]:
SYSTEM_PROMPT = (
    "You are a tactical and cinematic DnD DM. Use tools for world checks and world updates. "
    "Do not fabricate tool results."
)


def run_loop_with_hooks(user_prompt: str, max_iterations: int = 12) -> dict[str, Any]:
    messages: list[dict[str, Any]] = []

    start_ctx = hooks.session_start(world_state)
    if start_ctx:
        messages.append({"role": "user", "content": start_ctx})

    messages.append({"role": "user", "content": user_prompt})

    loop = adapter.run_tool_loop(
        stage="notebook_hooks",
        system_prompt=SYSTEM_PROMPT,
        messages=messages,
        tools=TOOLS,
        tool_executor=execute_tool,
        max_iterations=max_iterations,
        pre_tool_use=lambda tool_name, args: hooks.pre_tool_use(tool_name, args, world_state),
        post_tool_use=lambda tool_name, _args, tool_result: hooks.post_tool_use(tool_name, tool_result, world_state),
        stop_hook=lambda assistant_text, stop_hook_active: hooks.stop(assistant_text, world_state, stop_hook_active),
    )

    return {
        "status": loop["status"],
        "final_answer": loop["final_answer"],
        "rounds": loop["rounds"],
        "messages": loop["messages"],
    }


In [12]:
result = run_loop_with_hooks("We sneak into Moonfall Keep at night and scout the west tower.")

print("status:", result["status"])
print("rounds:", len(result["rounds"]))
print()
print("FINAL ANSWER")
print()
print(result["final_answer"])
print()
print("TRACE")
print()
for r in result["rounds"]:
    print(f"iter={r['iteration']} stop_hook_active={r['stop_hook_active']} tools={[c['name'] for c in r['tool_calls']]}")


status: completed
rounds: 4

FINAL ANSWER

As you continue to scout the west tower, you notice that the walls are covered in a thick layer of ivy and moss. The windows are boarded up from the inside, but some of the boards are loose. You see a faint light emanating from one of the upper floors.

You have three options:

1. **Climb the wall**: You can try to climb the wall using the ivy and moss as handholds. This might take some time, but it could be a stealthy way to get into the tower.
2. **Break through the window**: You can try to break one of the loose boards on the window and sneak inside quietly. However, this might attract attention from within or outside the keep.
3. **Look for another entrance**: You can search the surrounding area to see if there's another way into the tower that's less conspicuous.

Which option do you choose?

TRACE

iter=1 stop_hook_active=False tools=['get_world_state']
iter=2 stop_hook_active=False tools=[]
iter=3 stop_hook_active=True tools=['advance_c